<a href="https://colab.research.google.com/github/LonerNcoder/MechineL/blob/main/ML_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

My journey to learn mechine learning. It was my dream to learn various aspects of AI/Neural networking/Deep learning and its my first step towards my goal

***first model fashion mnist***

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from google.colab import files
from keras.preprocessing import image
import numpy as np
!pip install tensorflow-addons==0.16.1
import tensorflow_addons as tfa



In [ ]:
(training_images, training_labels),(test_images, test_labels) = tfds.as_numpy(tfds.load('fashion_mnist', split=['train', 'test'], batch_size=-1, as_supervised=True))

formating the images


In [3]:
training_images = training_images/255.0
test_images = test_images/255.0

In [4]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(training_images, training_labels, epochs=5)

In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
classify = model.predict(test_images)
print(classify[0])
print(test_labels[0])

*Second Model is Human or Horse*
*italicized text*

In [57]:
data = tfds.load('horses_or_humans', split='train', as_supervised=True)

def augmentimg(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/255)
  image = tf.image.random_flip_left_right(image)
  image = tfa.image.rotate(image, 40, interpolation='NEAREST')
  return image, label

data = data.map(augmentimg)
train_batches = data.shuffle(100).batch(10)

val_data = tfds.load('horses_or_humans', split='test', as_supervised=True)
val_data = val_data.map(augmentimg)
validation_batches = val_data.shuffle(50).batch(10)


from tensorflow.keras.callbacks import Callback

#creates a custom callback class
class CustomCallback(Callback):
    """
    This callback will stop the model from training once the model reaches 95% accuracy on the training data
    """
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_accuracy') > 0.95:
            print('Accuracy above 95% -- Stopping Training')
            self.model.stop_training = True #stops model training

my_callback = CustomCallback()

In [59]:
hoh_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [61]:
hoh_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [62]:
history = hoh_model.fit(train_batches, epochs=20, callbacks=my_callback,
                        validation_data=validation_batches, validation_steps=1)

103/103 [==============================] - 103s 988ms/step - loss: 0.0924 - accuracy: 0.9649 - val_loss: 1.1032e-04 - val_accuracy: 1.0000


In [67]:
test_acc = hoh_model.evaluate(validation_batches, verbose=0)[1]
print('Model Accuracy on Test Data:', round(test_acc,3))

Model Accuracy on Test Data: 0.863


In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300,300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  image_tensor = np.vstack([x])
  classes = hoh_model.predict(image_tensor)
  print(classes)
  print(classes[0])
